# Chapter 3: Keras and Data Retrieval in TensorFlow 2

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch03-Keras-and-Data-Retrieval/3.2.Creating_Input_Pipelines.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries and some setup

In [1]:
import os
import random
import numpy as np
import requests
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import tensorflow_datasets as tfds

def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
fix_random_seed(4321)
print("TensorFlow version: {}".format(tf.__version__))

data_dir = 'data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)


TensorFlow version: 2.12.0


## Using the `tf.data` API to retrieve data

Here we will be using the `tf.data` API to feed a dataset containing images of flowers. The dataset has a folder containing the images and a CSV file listing filenames and their corresponding label as an integer. We will write a TensorFlow data pipeline that does the following.

* Extract filenames and classes from the CSV
* Read in the images from the extracted filenames and resize them to 64x64
* Convert the class labels to one-hot encoded vectors
* Combine the processed images and one-hot encoded vectors to a single dataset
* Finally, shuffle the data and output as batches

### Downloading the data
The dataset is available at https://www.kaggle.com/olgabelitskaya/flower-color-images/data .

You need to download the zip file available in this URL and place it in the `data` folder in the `Ch03-Keras-and-Data-Retrieval` folder. You **do not** need to extract it as the following code will do it for you.

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d olgabelitskaya/flower-color-images -p /content/data

 94% 47.0M/50.1M [00:00<00:00, 81.1MB/s]
100% 50.1M/50.1M [00:00<00:00, 71.1MB/s]


In [6]:
# Section 3.2

import os
from zipfile import ZipFile

# Extracting the flowers image data to a directory
zip_filepath = [os.path.join('/content/data',f) for f in os.listdir('/content/data') if f.endswith('.zip')]

if len(zip_filepath)==0:
    print("Did you download the dataset as a zip file and place it in the Ch03-Keras-and-Data-Retrieval/data folder?")
elif len(zip_filepath)>1:
    print("There's too many .zip files. There should be only 1")

zfile = ZipFile(zip_filepath[0])
zfile.extractall('/content/data')

## Creating a tf.data.Dataset

Here we are creating the `tf.data` pipeline that executes the above steps.

In [7]:
os.path.sep

'/'

In [8]:
# Section 3.2
# Code listing 3.5

import tensorflow as tf
import os
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Read the CSV file with TensorFlow
# The os.path.sep at the end is important for the get_image function
data_dir = os.path.join('data', 'flower_images', 'flower_images') + os.path.sep
assert os.path.exists(data_dir)
csv_ds = tf.data.experimental.CsvDataset(
    os.path.join(data_dir,'flower_labels.csv') , ("",-1), header=True
)
# Separate the image names and labels to two separate sets
fname_ds = csv_ds.map(lambda a,b: a)
label_ds = csv_ds.map(lambda a,b: b)
print(label_ds)
def get_image(file_path):

    img = tf.io.read_file(data_dir + file_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [64, 64])

# Get the images by running get_image across all the filenames
image_ds = fname_ds.map(get_image)
print("The image dataset contains: {}".format(image_ds))
# Create onehot encoded labels from label data
label_ds = label_ds.map(lambda x: tf.one_hot(x, depth=10))
# Zip the images and labels together
data_ds = tf.data.Dataset.zip((image_ds, label_ds))

# Shuffle the data so that we get a mix of labels in every batch
data_ds = data_ds.shuffle(buffer_size= 20)
# Define a batch of size 5
data_ds = data_ds.batch(5)


<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
The image dataset contains: <_MapDataset element_spec=TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None)>


In [9]:
# Assuming label_ds is your label dataset
label_iterator = label_ds.as_numpy_iterator()

# Print label values
for label_value in label_iterator:
    print(label_value)


[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0

In [10]:
# Iterate through the data to see what it contains
for item in data_ds:
    print(item)
    break

(<tf.Tensor: shape=(5, 64, 64, 3), dtype=float32, numpy=
array([[[[0.14509805, 0.19117649, 0.11764707],
         [0.15196079, 0.20000002, 0.11960785],
         [0.15196079, 0.20098041, 0.11568628],
         ...,
         [0.17058825, 0.14705884, 0.14117648],
         [0.18039218, 0.17843139, 0.1529412 ],
         [0.22843137, 0.2245098 , 0.18039218]],

        [[0.14019608, 0.18529412, 0.11176471],
         [0.14607844, 0.18823531, 0.11470589],
         [0.15      , 0.2019608 , 0.11568628],
         ...,
         [0.23235296, 0.20098041, 0.19509806],
         [0.36176473, 0.327451  , 0.2911765 ],
         [0.37058824, 0.34411764, 0.29215688]],

        [[0.15196079, 0.19509806, 0.12352942],
         [0.14215687, 0.18235296, 0.10882354],
         [0.14117648, 0.18333334, 0.10980393],
         ...,
         [0.09901962, 0.10686275, 0.07941177],
         [0.12058824, 0.12254903, 0.09215687],
         [0.16862747, 0.18333334, 0.13333334]],

        ...,

        [[0.27450982, 0.36176473, 0

### Defining and training a model

Here we are defining a simple Convolution Neural Network (CNN) model to train it on the image data we just retrieved. You don't have to worry about the technical details of CNNs right now. We will discuss them in detail in the next chapter.

In [11]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a Convolution neural network for you to train for the flowers data
# We will discuss convolution neural networks in more detail later
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(64,64,3)),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Training the model with the tf.data pipeline
model.fit(data_ds, epochs=10)

Epoch 1/10
42/42 [==============================] - 11s 14ms/step - loss: 3.3972 - acc: 0.2238
Epoch 2/10
42/42 [==============================] - 0s 7ms/step - loss: 1.5740 - acc: 0.5095
Epoch 3/10
42/42 [==============================] - 0s 7ms/step - loss: 0.8457 - acc: 0.7619
Epoch 4/10
42/42 [==============================] - 0s 7ms/step - loss: 0.3499 - acc: 0.9143
Epoch 5/10
42/42 [==============================] - 0s 8ms/step - loss: 0.1351 - acc: 0.9762
Epoch 6/10
42/42 [==============================] - 0s 7ms/step - loss: 0.0268 - acc: 1.0000
Epoch 7/10
42/42 [==============================] - 0s 7ms/step - loss: 0.0159 - acc: 1.0000
Epoch 8/10
42/42 [==============================] - 0s 7ms/step - loss: 0.0145 - acc: 1.0000
Epoch 9/10
42/42 [==============================] - 0s 7ms/step - loss: 0.0077 - acc: 1.0000
Epoch 10/10
42/42 [==============================] - 1s 14ms/step - loss: 0.0020 - acc: 1.0000


## Using Keras data generators to retrieve data

Instead of `tf.data` API let us use the Keras `ImageDataGenerator` to retrieve the data. As you can see, the `ImageDataGenerator` involves much less code than the using the `tf.data` API.

In [12]:
# Section 3.2
# Code listing 3.6

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

data_dir = os.path.join('data','flower_images', 'flower_images')

# Defining an image data generator provided in Keras
img_gen = ImageDataGenerator()

# Reading the CSV files containing filenames and labels
labels_df = pd.read_csv(os.path.join(data_dir, 'flower_labels.csv'), header=0)

# Generating data using the flow_from_dataframe function
gen_iter = img_gen.flow_from_dataframe(
    dataframe=labels_df, directory=data_dir, x_col='file', y_col='label', class_mode='raw', batch_size=5, target_size=(64,64))

# Iterating through the data
for item in gen_iter:
    print(item)
    break

Found 210 validated image filenames.
(array([[[[ 47.,  51.,  36.],
         [ 41.,  52.,  24.],
         [ 36.,  53.,  20.],
         ...,
         [  6.,   6.,   6.],
         [  5.,   5.,   5.],
         [  5.,   5.,   5.]],

        [[ 48.,  57.,  42.],
         [ 66.,  74.,  58.],
         [ 37.,  38.,  28.],
         ...,
         [  4.,   4.,   4.],
         [  5.,   5.,   5.],
         [  5.,   5.,   5.]],

        [[ 31.,  41.,  23.],
         [ 35.,  40.,  29.],
         [ 52.,  52.,  43.],
         ...,
         [  4.,   4.,   4.],
         [  4.,   4.,   4.],
         [  5.,   5.,   5.]],

        ...,

        [[ 34.,  36.,  32.],
         [ 24.,  26.,  20.],
         [ 17.,  19.,  17.],
         ...,
         [ 16.,  18.,  15.],
         [  5.,   5.,   3.],
         [ 19.,  21.,  17.]],

        [[ 36.,  39.,  30.],
         [ 12.,  13.,  10.],
         [  7.,   7.,   7.],
         ...,
         [  9.,  11.,  10.],
         [  1.,   1.,   1.],
         [  8.,  10.,   8.]],

## Using the `tensorflow-datasets` library

Here we will use the `tensorflow-datasets` package. It is a curated list of popular datasets available for machine learning projects. With this package you can download a dataset in a single line. This means you don't have to worry about downloading/extracting/formatting data manually. All of that will be already done when you import data using the `tensorflow-datasets` library.

### Lists the available datasets

In [13]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow as tf
# See all registered datasets
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 'conll2003',
 'controlled_noisy_web_labels'

### Download the Cifar10 dataset and view information

In [14]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar10", with_info=True)
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete3ENI3J/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete3ENI3J/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path=PosixGPath('/tmp/tmpyvo4phe7tfds'),
    file_format=tfrecord,
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },


### Exploring the data

Here we will print the `data` and see what it provides. Then we will need to batch the data as data is provided as individual samples when you import it from `tensorflow-datasets`.

In [15]:
print(data)

{Split('train'): <_PrefetchDataset element_spec={'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>, Split('test'): <_PrefetchDataset element_spec={'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}


In [16]:
# Print some training data
train_ds = data["train"].batch(16)
for item in train_ds:
    print(item)
    break

{'id': <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'train_16399', b'train_01680', b'train_47917', b'train_17307',
       b'train_27051', b'train_48736', b'train_26263', b'train_01456',
       b'train_19135', b'train_31598', b'train_12970', b'train_04223',
       b'train_27152', b'train_49635', b'train_04093', b'train_17537'],
      dtype=object)>, 'image': <tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 

In [17]:
# Section 3.2

import tensorflow as tf

# Defining a dataset with batch size 16
train_ds = data["train"].batch(16)

# Creating a dataset that returns an (image, one-hot label) tuple
def format_data(x):
    return (x["image"], tf.one_hot(x["label"], depth=10))
train_ds = train_ds.map(format_data)

# Iterating the dataset
for item in train_ds:
    print(item)
    break

(<tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 123],
         [179, 142, 121],
         ...,
         [198, 163, 132],
         [201, 166, 135],
         [207, 174, 143]],

        [[187, 150, 117],
         [181, 143, 115],
         [175, 136, 113],
         ...,
         [201, 164, 132],
         [205, 168, 135],
         [207, 171, 139]],

        [[195, 161, 126],
         [187, 153, 123],
         [186, 151

### Training a simple CNN on the Cifar10 data

In [18]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a simple convolution neural network to process the CIFAR data
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(32,32,3)),
    Flatten(),
    Dense(10, activation='softmax')
])
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Fitting the model on the data for 25 epochs
model.fit(train_ds, epochs=25)

Epoch 1/25
3125/3125 [==============================] - 17s 5ms/step - loss: 3.1997 - acc: 0.1270
Epoch 2/25
3125/3125 [==============================] - 12s 4ms/step - loss: 2.2724 - acc: 0.1434
Epoch 3/25
3125/3125 [==============================] - 11s 4ms/step - loss: 2.2328 - acc: 0.1652
Epoch 4/25
3125/3125 [==============================] - 11s 4ms/step - loss: 2.2352 - acc: 0.1449
Epoch 5/25
3125/3125 [==============================] - 12s 4ms/step - loss: 2.2051 - acc: 0.1529
Epoch 6/25
3125/3125 [==============================] - 12s 4ms/step - loss: 2.1815 - acc: 0.1674
Epoch 7/25
3125/3125 [==============================] - 11s 4ms/step - loss: 2.1500 - acc: 0.1787
Epoch 8/25
3125/3125 [==============================] - 11s 4ms/step - loss: 2.1347 - acc: 0.1869
Epoch 9/25
3125/3125 [==============================] - 11s 3ms/step - loss: 2.1017 - acc: 0.2009
Epoch 10/25
3125/3125 [==============================] - 11s 4ms/step - loss: 2.0852 - acc: 0.2064
Epoch 11/25
3125/31